rasterio for reading .tif tiles and masks.

numpy for array operations.

imageio to save PNG overlays.

Path for easy file handling.

In [ ]:
# ================================
# IMPORT LIBRARIES
# ================================
import matplotlib.pyplot as plt
import rasterio
import numpy as np
from pathlib import Path
import imageio


Set the folders for:

Your input RGB tiles (TILE_DIR)

Prediction masks (PRED_DIR)

Where overlays will be saved (OUTPUT_DIR)

In [ ]:
# ================================
# CONFIGURATION
# ================================
# Update these paths to your folders
TILE_DIR = Path("PATH_TO_YOUR_INPUT_TILES")      # Folder with original RGB tiles
PRED_DIR = Path("PATH_TO_YOUR_PREDICTIONS")     # Folder with prediction masks
OUTPUT_DIR = PRED_DIR                           # Folder to save overlay PNGs

# Create output folder if it doesn't exist
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


Finds all .tif files in your input folder.

Sorts them to keep consistent order.

Prints how many tiles will be processed.

In [ ]:
# ================================
# GET LIST OF TILES
# ================================
# Collect all RGB tile file paths
tile_paths = sorted(TILE_DIR.glob("*.tif"))

print(f"Found {len(tile_paths)} tiles to process.")


Reads each RGB tile and its corresponding mask.

Converts RGB tile to [0-1] float array.

Overlays red [1,0,0] where the mask predicts a CAFO.

Converts back to uint8 (0–255) for PNG.

Saves PNG overlay in the output folder.

In [ ]:
# ================================
# PROCESS TILES AND CREATE OVERLAYS
# ================================
for tile_path in tile_paths:
    # Prediction mask path for this tile
    pred_path = PRED_DIR / f"{tile_path.stem}_prediction.tif"
    # Output PNG path
    overlay_path = OUTPUT_DIR / f"{tile_path.stem}_overlay.png"

    print(f"Processing {tile_path.name} → Saving overlay to {overlay_path.name}")

    # --- Load RGB tile ---
    with rasterio.open(tile_path) as src:
        rgb = src.read([1, 2, 3]).transpose(1, 2, 0).astype(np.float32) / 255.0  # H x W x 3

    # --- Load prediction mask ---
    with rasterio.open(pred_path) as src:
        mask = src.read(1)  # H x W binary

    # --- Overlay mask in red ---
    overlay = rgb.copy()
    overlay[mask == 1] = [1, 0, 0]  # Red color for predicted CAFOs

    # --- Convert to uint8 for saving ---
    overlay_uint8 = (overlay * 255).astype(np.uint8)

    # --- Save overlay ---
    imageio.imwrite(overlay_path, overlay_uint8)


Simple confirmation that all overlays were processed and saved.

In [ ]:

print(f"✅ All overlays saved to {OUTPUT_DIR}")
